
# Research on String Matching and Semantic Matching Techniques for Palimpsest

This notebook summarizes research on efficient algorithms for string matching and semantic matching techniques
applicable to the Palimpsest text analysis tool.
    


## Introduction

Palimpsest is a text analysis tool designed for comparing and analyzing large documents. This research explores
efficient algorithms for two key components:

1. **String Matching**: Techniques for identifying exact or approximate text matches across documents
2. **Semantic Matching**: Methods for finding content with similar meaning regardless of exact wording
    

In [ ]:

# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import Markdown, display

# Function to display markdown content
def md(text):
    display(Markdown(text))
    


## 1. String Matching Techniques

String matching algorithms find occurrences of exact or approximate patterns within text documents.
The following techniques have been analyzed for potential use in Palimpsest:
    

In [ ]:

# String matching algorithms comparison
algorithms = {
    "Exact Matching": [
        ("KMP (Knuth-Morris-Pratt)", "Linear time pattern matching", "O(m+n)", "Low", "Small-Medium"),
        ("Boyer-Moore", "Skips characters for efficiency", "O(m*n) worst, O(n/m) best", "Low", "Small-Medium"),
        ("Rabin-Karp", "Rolling hash function", "O(m*n) worst, O(m+n) average", "Medium", "Small-Medium"),
        ("Aho-Corasick", "Multiple pattern matching", "O(n + m + z)", "High", "Small-Medium")
    ],
    
    "Suffix Data Structures": [
        ("Suffix Trees", "Tree containing all suffixes", "O(n) construction, O(m) search", "High", "Small"),
        ("Suffix Arrays", "Sorted array of suffixes", "O(n log n) construction, O(m log n) search", "Medium", "Medium-Large"),
        ("Enhanced Suffix Arrays", "Suffix arrays with additional tables", "O(n) query after O(n) preprocessing", "Medium", "Medium-Large"),
        ("FM-Index", "Compressed full-text index", "Fast search with compressed storage", "High", "Large")
    ],
    
    "Approximate Matching": [
        ("Edit Distance (Levenshtein)", "Min operations to transform strings", "O(m*n)", "Low", "Small"),
        ("Bitap Algorithm", "Uses bit parallelism", "O(m*n/w) where w is word size", "Medium", "Small-Medium"),
        ("N-gram Overlap", "Compare shared n-grams", "O(m+n)", "Low", "Medium-Large"),
        ("Locality Sensitive Hashing", "Hash similar items to same buckets", "Dependent on implementation", "High", "Large")
    ]
}

# Display the algorithms with their properties
for category, techniques in algorithms.items():
    md(f"### {category}")
    
    table_data = []
    for name, desc, complexity, implementation, doc_size in techniques:
        table_data.append([name, desc, complexity, implementation, doc_size])
    
    df = pd.DataFrame(table_data, 
                     columns=["Algorithm", "Description", "Time Complexity", 
                              "Implementation Complexity", "Suitable Document Size"])
    display(df)
    

In [ ]:

# Example implementation of string matching using suffix arrays
def create_suffix_array(text):
    """
    Create a suffix array from text
    
    Args:
        text: Input text
        
    Returns:
        Sorted array of suffix indices
    """
    # Create list of all suffixes with their starting positions
    suffixes = [(i, text[i:]) for i in range(len(text))]
    
    # Sort by suffix
    suffixes.sort(key=lambda x: x[1])
    
    # Extract and return just the indices
    return [pos for pos, _ in suffixes]

def search_suffix_array(text, suffix_array, pattern):
    """
    Search for pattern in text using suffix array
    
    Args:
        text: Text to search in
        suffix_array: Suffix array of text
        pattern: Pattern to search for
        
    Returns:
        List of positions where pattern occurs
    """
    # Binary search for the pattern
    pattern_len = len(pattern)
    text_len = len(text)
    left, right = 0, len(suffix_array) - 1
    results = []
    
    # Find the first occurrence
    while left <= right:
        mid = (left + right) // 2
        suffix_pos = suffix_array[mid]
        suffix = text[suffix_pos:suffix_pos+pattern_len]
        
        if pattern == suffix:
            # Found a match, now find all matches
            results.append(suffix_pos)
            # Check to the left
            i = mid - 1
            while i >= 0:
                suffix_pos = suffix_array[i]
                suffix = text[suffix_pos:suffix_pos+pattern_len]
                if pattern == suffix:
                    results.append(suffix_pos)
                else:
                    break
                i -= 1
            # Check to the right
            i = mid + 1
            while i < len(suffix_array):
                suffix_pos = suffix_array[i]
                suffix = text[suffix_pos:suffix_pos+pattern_len]
                if pattern == suffix:
                    results.append(suffix_pos)
                else:
                    break
                i += 1
            break
        elif pattern < suffix:
            right = mid - 1
        else:
            left = mid + 1
    
    return sorted(results)

# Example usage
example_text = "banana"
suffix_array = create_suffix_array(example_text)
print(f"Text: {example_text}")
print(f"Suffix array: {suffix_array}")
print(f"Suffixes in sorted order:")
for i in suffix_array:
    print(f"  {i}: {example_text[i:]}")
    
pattern = "na"
matches = search_suffix_array(example_text, suffix_array, pattern)
print(f"Matches for '{pattern}': {matches}")

# For larger examples, we might use a library implementation
md("### Libraries for Efficient String Matching")
md("""
For production use in Palimpsest, consider these libraries:
- **Suffix Arrays**: `pysuffix` or `numpy` based implementations
- **FM-Index**: `pyFM` for Python implementation
- **Approximate Matching**: `fuzzywuzzy`, `rapidfuzz` or `difflib`
- **Multi-pattern**: `pyahocorasick` for Aho-Corasick implementation
""")
    


## 2. Semantic Matching Techniques

Semantic matching identifies textual content with similar meaning regardless of the exact words used.
This is essential for detecting conceptual similarities, paraphrasing, and translations.
    

In [ ]:

# Semantic matching techniques comparison
semantic_techniques = {
    "Classical Approaches": [
        ("TF-IDF + Cosine Similarity", "Compare document vectors", "Fast", "Low", "Limited semantic understanding"),
        ("LSA (Latent Semantic Analysis)", "SVD on term-document matrix", "Medium", "Medium", "Captures some semantic relationships"),
        ("LDA (Latent Dirichlet Allocation)", "Topic modeling", "Slow", "Medium", "Good for topic similarity")
    ],
    
    "Word Embeddings": [
        ("Word2Vec", "Neural word embeddings", "Fast once trained", "Medium", "Context-free word meaning"),
        ("GloVe", "Global word vectors", "Fast once trained", "Medium", "Global co-occurrence statistics"),
        ("FastText", "Subword embeddings", "Fast", "Medium", "Handles OOV words & morphology")
    ],
    
    "Contextual Embeddings": [
        ("BERT", "Bidirectional Transformer", "Slow", "High", "Context-dependent meaning"),
        ("RoBERTa", "Optimized BERT", "Slow", "High", "Improved training methodology"),
        ("DistilBERT", "Distilled BERT", "Medium", "Medium", "Faster with slight accuracy drop"),
        ("Sentence-BERT", "Sentence embeddings", "Medium", "Medium", "Optimized for sentence similarity")
    ],
    
    "Advanced Techniques": [
        ("Cross-Encoders", "Direct pair classification", "Very Slow", "High", "Highest accuracy for pairs"),
        ("Siamese Networks", "Neural text similarity", "Medium", "High", "Trained for similarity tasks"),
        ("Universal Sentence Encoder", "Sentence embeddings", "Medium", "Medium", "Multi-task trained embeddings"),
        ("MPNet", "Masked & permuted pre-training", "Slow", "High", "State-of-art embeddings")
    ]
}

# Display the semantic techniques with their properties
for category, techniques in semantic_techniques.items():
    md(f"### {category}")
    
    table_data = []
    for name, desc, speed, complexity, pros_cons in techniques:
        table_data.append([name, desc, speed, complexity, pros_cons])
    
    df = pd.DataFrame(table_data, 
                     columns=["Technique", "Description", "Processing Speed", 
                              "Implementation Complexity", "Quality"])
    display(df)
    

In [ ]:

# Example implementation of TF-IDF based semantic matching
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

def tfidf_similarity(documents):
    """
    Calculate pairwise TF-IDF cosine similarity between documents
    
    Args:
        documents: List of text documents
        
    Returns:
        Similarity matrix
    """
    # Create TF-IDF vectorizer
    vectorizer = TfidfVectorizer(stop_words='english')
    
    # Fit and transform the documents
    tfidf_matrix = vectorizer.fit_transform(documents)
    
    # Calculate cosine similarity
    similarity_matrix = cosine_similarity(tfidf_matrix, tfidf_matrix)
    
    return similarity_matrix

# Example 
example_docs = [
    "The quick brown fox jumps over the lazy dog",
    "A fast auburn fox leaps above the sleepy canine",
    "Python is a popular programming language",
    "Programming in Python is widely adopted in data science"
]

sim_matrix = tfidf_similarity(example_docs)

# Create DataFrame for better visualization
doc_names = [f"Doc {i+1}" for i in range(len(example_docs))]
sim_df = pd.DataFrame(sim_matrix, index=doc_names, columns=doc_names)
display(sim_df)

# Visualize the similarity matrix
plt.figure(figsize=(8, 6))
sns.heatmap(sim_df, annot=True, cmap='Blues')
plt.title('Document Similarity Matrix (TF-IDF)')
plt.tight_layout()
plt.show()

md("### Modern Semantic Matching Example")
md("""
For production implementation in Palimpsest, sentence transformers provide an excellent balance of accuracy and speed:
- **sentence-transformers**: Pre-trained models specifically for text similarity
- **HuggingFace Transformers**: Access to state-of-the-art language models
- **FAISS**: Fast similarity search for large document collections
- **Annoy**: Approximate nearest neighbors for efficient retrieval
""")
    


## 3. Recommendations for Implementation

Based on the research conducted, we recommend the following implementation approach for Palimpsest:

1. **Multi-tier Matching Strategy**:
   - Start with fast approximate methods (LSH, n-grams) to identify candidate matches
   - Apply more precise algorithms to promising candidates
   - Use semantic matching for concept-level similarity detection

2. **Scalability Considerations**:
   - Use data structures that support incremental updates (e.g., suffix arrays)
   - Implement document partitioning for parallel processing
   - Consider compression techniques for large document collections

3. **Hybrid Approach**:
   - Combine string and semantic matching for comprehensive document comparison
   - Weight different similarity metrics based on use case requirements
   - Allow users to configure precision vs. recall trade-offs
